In [48]:
# Install necessary packages
!pip install langchain langchain-openai faiss-cpu sentence-transformers transformers langchain-core --quiet
!pip install langchain_ollama langchain langchain-community langchain-core --quiet
!pip install ollama --quiet
!pip install pypdf --quiet


In [49]:
# --- Imports and Setup ---
import os
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [50]:
from langchain_community.document_loaders import PyPDFLoader

# List of PDF files
file_paths = ["/content/Faculty-Name-Short-Form.pdf", "/content/IUBAT-Department-Wise-Faculty-Information.pdf"]

# Load all documents
docs = []
for path in file_paths:
    loader = PyPDFLoader(path)
    docs.extend(loader.load())  # Append all pages from each PDF

In [51]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50
)
chunks = splitter.split_documents(docs)
print(f"Split into {len(chunks)} chunks")

Split into 128 chunks


In [ ]:
# --- 1. OpenAI Model Setup (via GitHub-hosted endpoint) ---
# NOTE: Secure your token. Never expose it in production or public code.
os.environ['GITHUB_TOKEN'] = "use_your_api_key"
token = os.environ.get("GITHUB_TOKEN")
endpoint = "https://models.github.ai/inference"
model_name = "openai/gpt-4.1-nano"

if not token:
    raise ValueError("GITHUB_TOKEN environment variable not set. Please provide a valid token.")

llm = ChatOpenAI(
    model_name=model_name,
    openai_api_key=token,
    openai_api_base=endpoint,
    temperature=0.5,
)


In [53]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
45 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as re

In [54]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [55]:
!ollama pull all-minilm

In [56]:
!pip install faiss-cpu

In [57]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

# Use Ollama with all-minilm embedding model (make sure Ollama is running)
embedding = OllamaEmbeddings(model="all-minilm")

faiss_db = FAISS.from_documents(docs, embedding=embedding)


# Save to disk
faiss_db.save_local("faiss_index")

# Load back
db = FAISS.load_local("faiss_index", embedding,allow_dangerous_deserialization=True)

In [58]:
!pip install rank_bm25

In [59]:
from rank_bm25 import BM25Okapi



# BM25 Setup
texts = [doc.page_content for doc in docs]
tokenized_corpus = [t.split(" ") for t in texts]
bm25 = BM25Okapi(tokenized_corpus)

query = "What is LangChain?"
bm25_scores = bm25.get_scores(query.split(" "))

# Get top N BM25 results
top_n = 2
bm25_ranks = sorted(enumerate(bm25_scores), key=lambda x: x[1], reverse=True)[:top_n]
bm25_docs = [docs[i[0]] for i in bm25_ranks]

# Get top FAISS results
faiss_docs = faiss_db.similarity_search(query, k=20)

# Merge & deduplicate
combined_docs = list({doc.page_content: doc for doc in (bm25_docs + faiss_docs)}.values())

In [60]:
from langchain.schema import Document

unique_pages = list({doc.page_content for doc in (bm25_docs + faiss_docs)})
combined_docs = [Document(page_content=content, metadata={"id": f"doc_{i}"}) for i, content in enumerate(unique_pages)]

vectorstore = FAISS.from_documents(combined_docs, embedding)

retriever = vectorstore.as_retriever()

In [61]:
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", "Given two uploaded PDFs (one containing all university faculty names with their short names, and the other containing faculty names with their mobile numbers, emails, and room numbers) and the latest student query which might reference information from these PDFs, formulate a standalone question that can be understood without needing the PDFs or chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

In [62]:
history_aware_retriever = create_history_aware_retriever(
    llm,retriever , contextualize_q_prompt
)

In [63]:
qa_prompt_with_memory = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\nContext: {context}"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])


In [64]:
qa_chain_with_memory = create_stuff_documents_chain(llm, qa_prompt_with_memory)
conversational_rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain_with_memory)

In [65]:
class SessionHistoryManager:
    def __init__(self):
        self.store = {}

    def get_history(self, session_id: str) -> ChatMessageHistory:
        if session_id not in self.store:
            self.store[session_id] = ChatMessageHistory()
        return self.store[session_id]

history_manager = SessionHistoryManager()

In [66]:
conversational_chain_with_history = RunnableWithMessageHistory(
    conversational_rag_chain,
    history_manager.get_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)


In [77]:

res1 = conversational_chain_with_history.invoke(
    {"input": "Give me the short name of Ms. Imrose Jahan."},
    config={"configurable": {"session_id": "user123"}

           }
)
print("Q:", "Give me the short name of Ms. Imrose Jahan.")
print("A:", res1['answer'])

Q: Give me the short name of Ms. Imrose Jahan.
A: The short name of Ms. Imrose Jahan is IJ.


In [78]:

res2 = conversational_chain_with_history.invoke(
    {"input": "What was my last question?"},
    config={"configurable": {"session_id": "user123"}

           }
)
print("Q:", "What was my last question?")
print("A:", res2['answer'])

Q: What was my last question?
A: Your last question was: "Give me the short name of Ms. Imrose Jahan."


In [76]:

res3 = conversational_chain_with_history.invoke(
    {"input": "Give me the short name of Dr Md Shariful Islam."},
    config={"configurable": {"session_id": "user123"}

           }
)
print("Q:", "Give me the short name of Dr Md Shariful Islam.")
print("A:", res3['answer'])

Q: Give me the short name of Dr Md Shariful Islam.
A: The short name of Dr Md Shariful Islam is DSI.
